# Similarity / Distance between Devices

In [16]:
from aux import *

import pandas as pd
import numpy as np

from stumpy import mass
from thefuzz import fuzz
from joblib import Parallel, delayed
from datetime import datetime, timedelta

import json

import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
import matplotlib.pyplot as plt
plt.style.use('default')

In [17]:
#Available colors
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

## Devices SDF definitions

### Adapt SDF definition data for analysis

In [18]:
# Load SDF files to compare
sdf_manager = SDFManager(path='../iot/sdf/')
sdfs, sdfs_dfs = sdf_manager.get_all_sdfs()
# Join sdf dfs into a single DataFrame
sdfs_df = pd.concat([value for key, value in sdfs_dfs.items()]).reset_index()

In [19]:
# Air Quality SDF DATAFRAME
sdfs_df[sdfs_df.thing=='AirQualityModified'].iloc[:,2:].reset_index(drop=True)

,thing_desc,obj,obj_desc,prop,prop_desc,prop_type,prop_unit


## Devices data

### Load devices data

In [49]:
# Read devices data samples from JSON
with open('devices.json', 'r') as f: devices = json.load(f)

### Adapt devices data for analysis

In [50]:
# Build devices dataframe
rows = []
for dev_uuid, dev in devices.items() :
    # Dev row initialization
    row = {
        'uuid': dev_uuid,
        'class' : dev['class'],
        'integ': dev['integrated'],
        'period': dev['period']
    }
        
    # Create a row for each module attribute with a column for each value in the buffer
    for mod_name, attribs_dic in dev['modules'].items() :
        row['mod'] = mod_name
        for attrib_name, values in attribs_dic.items() :
            row['attrib'] = attrib_name
            for i, val in enumerate(values) : row[f'v{i+1}'] = val
            rows.append(row.copy())

# Build devices dataframe
devs_df = pd.DataFrame(rows)

## Similarity Analysis
Given an unknown device, determine the 5 closest devices according to SDF definition similarity, and then choose the closest device by including the buffered data. 

**Possible algorithm**

    Given an unknown device(thing) module(object) attribute(property), find 5 closest thing-object-property tuples according to SDF definition.

    Then, for each unknown module attribute, determine which attribute resembles the most to it of those in the 5 closest devices, and vote this device UUID as the candidate to be the closest.

    Finally, the device that has the most attribute votes is chosen as the closest device.

In [51]:
# Value columns names
val_cols = devs_df.columns[6:]

# Turn booleans into integers and cast all data to float
devs_df[val_cols] = devs_df[val_cols].replace('False',0).replace('True',1)
devs_df[val_cols] = devs_df[val_cols].astype(float)

In [52]:
# Air Quality Modified DEVICES DATAFRAME
devs_df[devs_df['class']=='AirQualityModified']

,uuid,class,integ,period,mod,attrib,v1,v2,v3,v4,...,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28


In [53]:
# Unknown class
dev_class = 'AirQualityModified'

In [54]:
# Compute voting results df
def calc_voting_result_df(votes) :
    total_vote_sdf = {}
    for vote in votes:
        for candidate, score in vote.items() :
            if candidate not in total_vote_sdf :
                total_vote_sdf[candidate] = score
            else :
                total_vote_sdf[candidate] += score

    return pd.DataFrame(total_vote_sdf.items(),columns=['candidate','score']).sort_values(by='score',ascending=False)

### Determine closest devices by SDF comparison

String similarity of the fields:
- Thing and its description
- Object and its description
- Property and its description

In [55]:
noninteg_class = sdfs_df[sdfs_df.thing == dev_class]
integ_classes = sdfs_df[sdfs_df.thing != dev_class]

In [56]:
# Define distance functions
def calc_str_dist(descs, row):
    return fuzz.ratio(descs,row['prop'] + ' ' + row['prop_desc'])

# Compute closest classes by comparing SDF descriptions
def get_closest_classes(noninteg_class,integ_classes,i,score=3) :
    # Create local copies and compare only rows with same data type
    noninteg_class_row = noninteg_class.iloc[i].copy()
    integ_classes = integ_classes[integ_classes.prop_type==noninteg_class_row['prop_type']].copy()

    # Build non integrated row text description
    non_integ_class_row_desc = noninteg_class_row['prop'] + ' ' + noninteg_class_row['prop_desc']

    # Calc string distances to each other integrated row text description
    integ_classes['str_dist'] = integ_classes.apply(lambda x: calc_str_dist(non_integ_class_row_desc,x), axis=1)
    closest_things = integ_classes[['thing','obj','prop','str_dist']].sort_values(by='str_dist',ascending=False)

    # Give points based on closeness
    vote = {}
    for row in closest_things.itertuples() :
        if score == 0 : break
        if row.thing in vote : continue
        vote[row.thing] = score
        score -= 1

    return vote

In [57]:
# Compute Top 5 closest SDF classes
votes = (Parallel(n_jobs=12)(delayed(get_closest_classes)(noninteg_class,integ_classes,i) for i in range(noninteg_class.shape[0])))
voting_result_df = calc_voting_result_df(votes)
closest_classes = voting_result_df.candidate.iloc[0:5].tolist()
voting_result_df

,candidate,score


### Compare AirQuality Devices

In [58]:
# Select data to plot
dev1_uuid = 'indoors_airquality'
dev1_df = devs_df[devs_df.uuid == dev1_uuid]
dev1_timestamps = [datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S.%f") for timestamp in devices[dev1_uuid]['timestamps']]

dev2_uuid = 'outdoors_airquality'
dev2_df = devs_df[devs_df.uuid == dev2_uuid]
dev2_timestamps = [datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S.%f") for timestamp in devices[dev2_uuid]['timestamps']]

#dev3_uuid = 'indoors_airqualitysimp'
#dev3_df = devs_df[devs_df.uuid == dev3_uuid]
#dev3_timestamps = [datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S.%f") for timestamp in devices[dev3_uuid]['timestamps']]

devs = [dev1_df,dev2_df]
ts = [dev1_timestamps,dev2_timestamps]

In [59]:
names = []
fig = go.Figure()
for i, (dev, t) in enumerate(zip(devs,ts)) :
    for j, row in dev.iterrows() :
        dev_uuid = row.uuid
        dev_class, mod_name, attrib_name = row['class'], row['mod'], row['attrib']
        if mod_name in  ['air_quality_sensor','pressure_sensor'] : continue
        name = row.uuid + '/' + mod_name + '/' + attrib_name
        fig.add_trace(go.Scatter(x=t, y=row[val_cols],
                    mode='lines',
                    name=name))
        names.append(name)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.8,
    xanchor="left",
    x=0.55
))
fig.show()

### Compare PickUpRobot Devices

In [62]:
# Select data to plot
dev1_uuid = 'bodyconfig_pickuprob'
dev1_df = devs_df[devs_df.uuid == dev1_uuid]
dev1_timestamps = [datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S.%f") for timestamp in devices[dev1_uuid]['timestamps']]

dev2_uuid = 'windowmilling_pickuprob'
dev2_df = devs_df[devs_df.uuid == dev2_uuid]
dev2_timestamps = [datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S.%f") for timestamp in devices[dev2_uuid]['timestamps']]

#dev3_uuid = 'bodyconfig_pickuprob2'
#dev3_df = devs_df[devs_df.uuid == dev3_uuid]
#dev3_timestamps = [datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S.%f") for timestamp in devices[dev3_uuid]['timestamps']]

devs = [dev1_df,dev2_df]
ts = [dev1_timestamps,dev2_timestamps]

In [65]:
names = []
fig = go.Figure()
for i, (dev, t) in enumerate(zip(devs,ts)) :
    for j, row in dev.iterrows() :
        dev_uuid = row.uuid
        dev_class, mod_name, attrib_name = row['class'], row['mod'], row['attrib']
        if attrib_name not in  ['x_position'] : continue
        name = row.uuid + '/' + mod_name + '/' + attrib_name
        fig.add_trace(go.Scatter(x=t, y=row[val_cols],
                    mode='lines',
                    name=name))
        names.append(name)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.55
))
fig.show()

### Determine closest device UUID by buffered values comparison

Get the attribute with the closest values (euclidean distance) to each of the unknown device module attributes

In [71]:
pickup_uuid = 'bodyconfig_pickuprob2'
airqualitymod_uuid = 'indoors_airqualitymod'
uuid = airqualitymod_uuid # UUID of the unknown device

In [72]:
noninteg_dev = devs_df[devs_df.uuid == uuid]
integ_devs = devs_df[(devs_df.integ == True) & (devs_df.uuid != uuid)]

In [73]:
# Compute closest devices searching for closest time series pattern
def get_closest_devs(noninteg_dev,integ_devs,closest_classes,i,score=1) :
    # Create local copies
    noninteg_dev_row = noninteg_dev.iloc[i].copy()
    closest_classes.append(noninteg_dev_row.dev)
    integ_devs = integ_devs[integ_devs.dev.isin(closest_classes)].copy()
    val_cols = integ_devs.columns[6:]

    # Compute device with closest time series pattern
    min_dist_profile = np.Inf
    query_series = noninteg_dev_row[val_cols[:20]].astype(float).to_numpy()
    for i, integ_dev_row in integ_devs.iterrows() :
        inspected_series = integ_dev_row[val_cols].dropna().astype(float).to_numpy()
        if inspected_series.size < query_series.size : continue
        # MASS Distance Profile
        dist_profile = mass(query_series, inspected_series, normalize=False)
        if np.min(dist_profile) < min_dist_profile :
            min_dist_profile = np.min(dist_profile)
            candidate = integ_dev_row.dev + '/' + integ_dev_row.uuid
    
    # The winner is the one with lower distance
    return {candidate: score}
    

In [74]:
# Out of those 5 closest classes, get device that best matches time series pattern
votes = (Parallel(n_jobs=12)(delayed(get_closest_devs)(noninteg_dev,integ_devs,closest_classes,i) for i in range(noninteg_dev.shape[0])))
voting_result_df = calc_voting_result_df(votes)

In [75]:
voting_result_df

,candidate,score
0,AirQuality/indoors_airquality,4


## USE SIMILARITY ANALYSIS TO SOLVE CASES. 
### Case 1. A KNOWN DEVICE DISAPPEARS AND A NEW ONE WITH SIMILAR CHARACTERISTICS APPEARS
### Case 2. A COMPLEMENTARY DEVICE APPEARS IN A TASK

Explain what needs to be done to be able to tackle CASE 3. What others steps should we take?
Additionally, explain the weaknesses of our similarity comparison and how could these be tackled.